In [1]:
# !pip install -U datasets

In [2]:
# !sudo -H pip install -q transformers --upgrade

In [5]:
# !pip uninstall autoawq -y

In [ ]:
# !pip install autoawq

In [3]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [4]:
# !pip install transformers==4.35.2

In [1]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [2]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [8]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
import gc
gc.collect()


31

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [41:02<00:00, 76.96s/it]


In [ ]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/llmmodels/quantized_model/awq_mistral/tokenizer_config.json',
 '/llmmodels/quantized_model/awq_mistral/special_tokens_map.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.model',
 '/llmmodels/quantized_model/awq_mistral/added_tokens.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.json')

### Inference on quantized model

In [3]:
from transformers import pipeline
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [4]:
model_id = quant_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")

In [12]:
import time
start = time.time()
output2 = pipe("I love data science",
               max_new_tokens=500,
               num_return_sequences=1)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 20.34541630744934


In [11]:
output2

'the long walks we used to had.\n\nI’m not sure if I’m ready to let go of the past. I’m not sure if I’m ready to let go of the memories. I’m not sure if I’m ready to let go of the pain. I’m not sure if I’m ready to let go of the love.\n\nI’m not sure if I’m ready to let go of you.\n\nI’m not sure if I’m ready to let go of me.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anything.\n\nI’m not sure if I’m ready to let go of anythi

### Inference on un-quantized model

In [3]:
from transformers import pipeline
# do_sample=True,
# top_k=50,
# top_p=0.95

In [5]:
model_id = model_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
# Using unquant model
import time
start = time.time()
output1 = pipe("how deep learning is different from machine learning",
               max_new_tokens=500,
               num_return_sequences=1)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 50.44006586074829


In [37]:
output1

'how deep learning is different from machine learning.\n\n## What is Machine Learning?\n\nMachine learning is a subset of artificial intelligence that allows computers to learn from data without being explicitly programmed. It is a method of data analysis that automates analytical model building. It is a branch of computer science that gives computers the ability to learn without being explicitly programmed.\n\nMachine learning algorithms build a mathematical model based on sample data, known as “training data,” in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the task.\n\nMachine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop convent

In [27]:
output1 == output2

True